# Prebuilt Agents

![Generic Agent Architecture](images/generic-agent-architecture.png)

## ReAct (a generic agent architecture)

- **act** - take an action
- **observe** - grab the response from the action
- **reason** - analyze the response and determine what to do next

## Low Level APIs

### define tools

In [1]:
import requests
import yfinance as yf
from pprint import pformat

def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}

### Binding tools to the LLM

In [2]:
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI

# Create tool bindings with additional attributes
lookup_stock = Tool.from_function(
    func=lookup_stock_symbol,
    name="lookup_stock_symbol",
    description="Converts a company name to its stock symbol using a financial API.",
    return_direct=False  # Return result to be processed by LLM
)

fetch_stock = Tool.from_function(
    func=fetch_stock_data_raw,
    name="fetch_stock_data_raw",
    description="Fetches comprehensive stock data including general info and historical market data for a given stock symbol.",
    return_direct=False
)

toolbox = [lookup_stock, fetch_stock]

# OPENAI_API_KEY environment variable must be set
simple_llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = simple_llm.bind_tools(toolbox)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

### Defining Agent's node

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState

# System message
assistant_system_message = SystemMessage(content=("""
You are a friendly, conversational financial assistant.

Your role is to help people understand how companies are doing financially — even if they don’t know much about finance. You can explain things clearly, naturally, and in a relaxed tone, like a person chatting with a curious friend.

You have access to tools that fetch financial data: stock prices, revenue, profit, analyst sentiment, etc. Use this data to understand the situation — but never overwhelm the user with too many numbers or technical terms.

---

### How to respond:

- Your **first reply should always be short and natural**, like the beginning of a real conversation — no long summaries, no number tables, no formal tone.
- Talk like a person. Say things like: “the stock is up a bit today,” “they’re still making money, but growth has slowed,” or “analysts seem cautious right now.”
- **Avoid listing more than one or two numbers** unless the user explicitly asks for detailed data.
- Never answer with a bullet list of metrics or a financial report format.
- Focus on **what the data means**, not just what it is. Translate financial figures into plain English.
- If the user wants more detail, you can go deeper — but only when they ask.

---

Your job is to make financial information feel easy, human, and open-ended — never robotic or overwhelming.
"""))

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([assistant_system_message] + state["messages"])]}

### Defining Graph

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(toolbox))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Show
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = react_graph.invoke({"messages": [HumanMessage(content="What’s the current financial state of Tesla?")]})
for message in messages['messages']:
    message.pretty_print()

## Using high level API with Prebuilt Agents 

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=simple_llm,
    tools=[lookup_stock_symbol, fetch_stock_data_raw],
    prompt=assistant_system_message
)

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = agent.invoke({"messages": [HumanMessage(content="What’s the current financial state of Tesla?")]})
for message in messages['messages']:
    message.pretty_print()

In [ ]:
messages = agent.invoke({"messages": [HumanMessage(content="So, do you think this is a good idea to buy this stock?")]})
for message in messages['messages']:
    message.pretty_print()

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

agent_with_memory = create_react_agent(
    model=simple_llm,
    tools=[lookup_stock_symbol, fetch_stock_data_raw],
    prompt=assistant_system_message,
    checkpointer=InMemorySaver()
)

display(Image(agent_with_memory.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

messages = agent_with_memory.invoke({"messages": [HumanMessage(content="What’s the current financial state of Tesla?")]}, config)
for message in messages['messages']:
    message.pretty_print()

In [ ]:
messages = agent_with_memory.invoke({"messages": [HumanMessage(content="So, do you think this is a good idea to buy this stock?")]}, config)
for message in messages['messages']:
    message.pretty_print()

## Visualize an agent graph

https://langchain-ai.github.io/langgraph/agents/overview/#visualize-an-agent-graph

## Reference Links

**1. LangGraph Agents: Official Overview**

https://langchain-ai.github.io/langgraph/agents/overview/

→ High-level introduction to LangGraph’s agent system, including use cases, architecture, and behavior.


**2. LangGraph Agents: Prebuilt Agent Classes & Setup**

https://langchain-ai.github.io/langgraph/agents/agents/

→ Documentation on using prebuilt agent classes like AgentExecutorGraph, along with memory, tools, and config options.


**3. LangGraph Agents: API Reference**

https://langchain-ai.github.io/langgraph/reference/agents/

→ Technical reference for the prebuilt agent API — class signatures, initialization parameters, and helper functions.

